In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('data/COVID19MEXICO.csv')

In [4]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,INDIGENA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2024-12-31,0aa89f,1,10,9,2,15,9,3,1,2024-01-26,2024-01-25,9999-99-99,97,2,31,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,1,5,5,2,97,7,7,99,México,97,97
1,2024-12-31,g3d9fdf,1,12,9,1,15,9,3,1,2024-01-01,2024-01-01,9999-99-99,97,2,1,1,2,99,99,98,98,98,98,98,98,98,98,98,98,99,2,997,997,2,97,6,6,99,México,97,97
2,2024-12-31,g9637c9,1,4,25,1,25,25,12,2,2024-01-02,2024-01-01,9999-99-99,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,10,998,2,97,7,7,99,México,97,2
3,2024-12-31,g824984,1,4,32,2,32,32,17,2,2024-01-02,2024-01-02,9999-99-99,2,2,2,1,97,2,2,2,2,2,2,2,2,2,1,2,2,2,1,34,29,2,97,3,7,99,México,97,2
4,2024-12-31,gdb0743,1,3,19,2,19,19,39,1,2024-01-02,2024-01-02,9999-99-99,97,2,52,1,97,2,2,2,2,2,2,2,2,2,2,2,2,2,1,5,5,2,97,7,7,99,México,97,97


Se seleccionan unicamente los registros que sean confirmados y que sean casos positivos de COVID-19

Según el catalogo de datos, los valores 1,2,3 son casos confirmados 

In [5]:
df["CLASIFICACION_FINAL_COVID"].value_counts()

CLASIFICACION_FINAL_COVID
7    86947
6    70932
3    14103
5     3480
4      124
Name: count, dtype: int64

In [6]:
df = df[df["CLASIFICACION_FINAL_COVID"] == 3]

### Se crea una nueva variable que es saber si la persona falleció o no

In [7]:
df["DEFUNCION"] = np.where(df['FECHA_DEF'] == '9999-99-99', 2, 1)

### Se pasa a formato fecha

#### Primero se pasa a nulos los valores de 9999-99-99, que representan lo que no han fallecido

In [8]:
df['FECHA_DEF'] = df['FECHA_DEF'].replace('9999-99-99', np.nan)

In [9]:
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'])

In [10]:
df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])

Vamos a calcular el numero de dias que han pasado desde la fecha de ingreso/sintomas hasta la fecha de defuncion

In [11]:
df['DIAS_INGRESO_DEF'] = (df['FECHA_DEF'] - df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS_DEF'] = (df['FECHA_DEF'] - df['FECHA_SINTOMAS']).dt.days

In [12]:
df[df['DEFUNCION']== 1][['DIAS_INGRESO_DEF', 'DIAS_SINTOMAS_DEF']]

,DIAS_INGRESO_DEF,DIAS_SINTOMAS_DEF
445,15.0,16.0
766,8.0,9.0
2202,4.0,9.0
2423,0.0,0.0
2514,4.0,4.0
...,...,...
166264,1.0,5.0
168345,2.0,4.0
168635,4.0,5.0
168747,1.0,1.0


Seleccionamos las columnas que nos interesan para el análisis

In [ ]:
df_cox = df[["DEFUNCION","DIAS_INGRESO_DEF","DIAS_SINTOMAS_DEF", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "OTRA_COM", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO"]]
# df_cox = df[["DEFUNCION","DIAS_INGRESO_DEF","DIAS_SINTOMAS_DEF", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "OTRA_COM", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO", "SEXO"]]

In [14]:
df_cox.head()

,DEFUNCION,DIAS_INGRESO_DEF,DIAS_SINTOMAS_DEF,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,SEXO
3,2,NaN,NaN,2,2,2,2,2,2,2,2,1,2,2,2
5,2,NaN,NaN,2,2,2,2,2,2,2,2,2,2,2,1
16,2,NaN,NaN,2,2,2,2,2,2,2,2,2,2,2,1
19,2,NaN,NaN,2,2,2,2,2,2,2,2,2,2,2,1
22,2,NaN,NaN,2,2,2,2,2,2,2,2,2,2,2,2


In [16]:
df_cox["DEFUNCION"].value_counts()

DEFUNCION
2    13273
1      830
Name: count, dtype: int64

Camabiar valores para que 1 sea si (prensenta condicion o afirmacion de nombre de columna) y 0 sea no (no presenta condicion o negación de nombre de columna)

In [17]:
df_cox.replace([1,2], [1,0], inplace=True)

C:\Users\daniel\AppData\Local\Temp\ipykernel_13680\3344868473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox.replace([1,2], [1,0], inplace=True)


Vamos a eliminar los registros con valores nulos

In [18]:
day_variables = df_cox[["DIAS_INGRESO_DEF","DIAS_SINTOMAS_DEF"]]

In [19]:
df_cox = df_cox.drop(day_variables, axis=1)

In [20]:
df_cox.replace([97,98,99], np.nan, inplace=True)

In [21]:
df_cox = df_cox.dropna()

In [22]:
df_cox["DEFUNCION"].value_counts()

DEFUNCION
0    13185
1      818
Name: count, dtype: int64

In [23]:
df_cox = df_cox.astype(int)

In [24]:
df_cox = df_cox.join(day_variables, how='left')

In [ ]:
# df.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_PCR',
       'RESULTADO_PCR_COINFECCION', 'TOMA_MUESTRA_ANTIGENO',
       'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL_COVID',
       'CLASIFICACION_FINAL_FLU', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'DEFUNCION', 'DIAS_INGRESO_DEF',
       'DIAS_SINTOMAS_DEF'],
      dtype='object')

In [ ]:
# df_dashboard = df[['SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
#        'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
#        'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'INDIGENA',
#        'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
#        'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO','MIGRANTE', 'PAIS_NACIONALIDAD',
#        'PAIS_ORIGEN', 'UCI', 'DEFUNCION', 'DIAS_INGRESO_DEF',
#        'DIAS_SINTOMAS_DEF']]

In [ ]:
# df_dashboard.to_csv('data/df_dashboard.csv', index=False)

In [ ]:
# df_cox.to_csv('data/df_cox.csv', index=False)

In [25]:
df_cox.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14003 entries, 3 to 175112
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DEFUNCION          14003 non-null  int32  
 1   NEUMONIA           14003 non-null  int32  
 2   DIABETES           14003 non-null  int32  
 3   EPOC               14003 non-null  int32  
 4   ASMA               14003 non-null  int32  
 5   INMUSUPR           14003 non-null  int32  
 6   HIPERTENSION       14003 non-null  int32  
 7   OTRA_COM           14003 non-null  int32  
 8   CARDIOVASCULAR     14003 non-null  int32  
 9   OBESIDAD           14003 non-null  int32  
 10  RENAL_CRONICA      14003 non-null  int32  
 11  TABAQUISMO         14003 non-null  int32  
 12  SEXO               14003 non-null  int32  
 13  DIAS_INGRESO_DEF   818 non-null    float64
 14  DIAS_SINTOMAS_DEF  818 non-null    float64
dtypes: float64(2), int32(13)
memory usage: 1.5 MB


In [26]:
df_cox.columns

Index(['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'SEXO', 'DIAS_INGRESO_DEF',
       'DIAS_SINTOMAS_DEF'],
      dtype='object')

Se optó por agregar +1 dia a cada registro en las columnas de dias, ya que existen pacientes que fallecieron el mismo dia, por lo que su valor era 0. La regresion cox toma el 0 como un valor representativo de que no ocurrio el evento, entonces por esa razon se tomó la decision de agregar un dia a cada registro.

No se agrego solo a los registros con valor 0 sino un dia a todos los registros con el fin de:

Mantienes la distribución original sin favorecer a los pacientes con 1 día, ya que aumentaria su numero de registros.
Evitas tiempos cero, garantizando que el modelo de Cox funcione sin problemas.
Preservas la representación relativa de los tiempos de evento y censura.

In [27]:
df_cox["DIAS_SINTOMAS_DEF"] = df_cox["DIAS_SINTOMAS_DEF"]+1

In [28]:
df_cox["DIAS_INGRESO_DEF"] = df_cox["DIAS_INGRESO_DEF"]+1

Rellenamos los nulos con valor 0

In [29]:
df_cox.fillna(0, inplace=True)

In [30]:
df_cox = df_cox.astype(int)

In [ ]:
df_cox.to_csv('data/df_cox.csv', index=False)

In [ ]:
# df_cox_hombre = df_cox[df_cox["SEXO"] == 0]
# df_cox_mujer = df_cox[df_cox["SEXO"] == 1]

In [ ]:
# df_cox_hombre.to_csv('data/df_cox_hombre.csv', index=False)
# df_cox_mujer.to_csv('data/df_cox_mujer.csv', index=False)